# TabularDNN

Use `/usr/bin/python3.11 -m venv venvName` to make venv. Use `source venvName/bin/activate` to enter venv. Python will be 3.11 when used either `python` or `python3`. To get out of venv use `deactivate`. `rm -r venvName` to delete venv.

Use `pip install tensorflow[and-cuda]` to install and check with `python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"` to see if GPU is set up. If GPU devices are shown then everything is OK.

Use `pip install jupyter` to have jupyter-notebook in venv. Check with `which jupyter` to see if correct jupyter is used. Use `python -m ipykernel install --user --name=venvName` to add venv to jupyter kernels. When creating new notebook instead of using python3 use venvName. To remove venv kernel from jupyter use `jupyter kernelspec remove venvName`

In [1]:
! python --version

Python 3.11.7


In [2]:
import tensorflow as tf
import keras

2024-10-18 18:49:09.132161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 18:49:09.147872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 18:49:09.158685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 18:49:09.161882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 18:49:09.170336: I tensorflow/core/platform/cpu_feature_guar

In [3]:
print(f"Keras Version: {keras.__version__}")
print(f"TensorFlow Version: {tf.__version__}")

Keras Version: 3.6.0
TensorFlow Version: 2.17.0


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("dataset/AQI.csv")
df.head()

,Date,Time,Temperature,Real Feel,Weather Status,Wind Gusts,Humidity,Indoor Humidity,Humidity Status,Dew Point,...,AQI,AQI-PM2.5,AQI-PM10,AQI-NO2,AQI-O3,Amount-PM2.5,Amount-PM10,Amount-NO2,Amount-O3,Filename
0,"THURSDAY, 17 OCTOBER",17:30,11,12,Mostly cloudy,6,0.80,0.46,Ideal Humidity,8,...,37,13,14,37,15,3,10,18,45,Seattle/95.png
1,"THURSDAY, 10 OCTOBER",09:00,11,13,Mostly cloudy,8,0.92,0.50,Ideal Humidity,9,...,39,12,14,39,7,3,10,19,19,Seattle/44.png
2,"TUESDAY, 8 OCTOBER",13:06,16,16,Cloudy,11,0.93,0.72,Slightly Humid,15,...,45,17,18,45,6,4,13,22,18,Seattle/6.png
3,"THURSDAY, 17 OCTOBER",18:28,10,12,Partly cloudy,5,0.82,0.44,Ideal Humidity,7,...,50,21,22,50,11,5,17,26,33,Seattle/96.png
4,"WEDNESDAY, 9 OCTOBER",16:53,16,18,Partly sunny,8,0.68,0.53,Ideal Humidity,10,...,31,20,31,30,15,4,26,15,45,Seattle/25.png
